# Chapter 6 – Life: Moore's Multiform *Byron*

The final chapter of *Romanticism and the Contingent Self* is devoted to Thomas Moore's [*Letters and Journals of Lord Byron, with Notices of his Life*](https://www.lordbyron.org/contents.php?doc=ThMoore.1830.Contents) (1830-31), which was recognised in its day as a classic of literary biography. As in previous chapters, the aim is twofold. First, I seek to integrate a genre of 'contingent selfhood' into a general morphology of Romantic literature. Second, I see to establish appropriate aesthetic standards for judging 'formless' works of contingent selfhood. In this case, I focus on only a single case study. Moore's *Byron* is a vast text, and incorporates both 'monstrous' and 'multiple' portrayals of the self. Only a small number of literary biographies are available for comparison, so the analysis uses a smaller corpus.

In [42]:
from romanticself import BiographyCorpus, Biography, XMLBiography
from nltk import word_tokenize
import pandas as pd
import numpy as np
import rpy2
import seaborn as sns
corpus = BiographyCorpus("data/biography-corpus/", tokenizer = word_tokenize)
moore = corpus.data["moore.xml"]

# Section 6.2 Plot: Ordered and Disordered lives

In the first section I analyse Moore's prose style.

What is the average sentence length for each biographer?


In [30]:
def biography_stats(biography: Biography) -> dict:
    return {
        "Author": biography.author,
        "Date": biography.date,
        "Title": biography.title,
        "Word count": len([token for token in biography.iter_tokens(allowed_authors = None)]),
        "Mean sentence length": np.mean([len(word_tokenize(sent.text)) for sent in biography.sentences])
    }

figure_6_1 = pd.DataFrame.from_records([biography_stats(bio) for bio in corpus])
figure_6_1.to_csv("figures/figure_6_1.csv")
display(figure_6_1)

,Author,Date,Title,Word count,Mean sentence length
0,John Galt,1830,The Life of Lord Byron,114742,36.989684
1,Elizabeth Gaskell,1857,The Life of Charlotte Brontë,209379,27.288414
2,John Gibson Lockhart,1837,"Memoirs of the Life of Sir Walter Scott, Bart.",999354,35.088445
3,Thomas Medwin,1847,The Life of Percy Bysshe Shelley,144310,38.167152
4,Thomas Moore,1830,Letters and Journals of Lord Byron,636609,33.342586
5,Robert Southey,1813,The Life of Nelson,112030,33.164594


How are the words distributed in the two lives-and-letters biographies?

In [51]:
# The subjects of the two lives-and-letters biographies
# NB: Neither appears as author in the other biography
subjects = {"LdByron", "WaScott"}
sub_corpus = [val for key,val in corpus.data.items() if key in {"moore.xml", "lockhart.xml"}]

def lives_and_letters_data(bio: Biography):
    return {
        "Title": bio.title,
        "Author": len([token for token in bio.iter_tokens(allowed_authors=bio.author_id)]),
        "Subject": len([token for token in bio.iter_tokens(allowed_authors=subjects)]),
        "Other": len([token for token in bio.iter_tokens(allowed_authors=set(bio.all_authors) - {bio.author_id} - subjects)])
    }

figure_6_2_data = pd.DataFrame.from_records(lives_and_letters_data(bio) for bio in sub_corpus)
display(figure_6_2_data)

,Title,Author,Subject,Other
0,"Memoirs of the Life of Sir Walter Scott, Bart.",466072,533282,0
1,Letters and Journals of Lord Byron,260671,368442,7496


In [60]:
pd.melt(figure_6_2_data, id_vars = "Title", var_name = "Author", value_name="Word count")

,Title,Author,Word count
0,"Memoirs of the Life of Sir Walter Scott, Bart.",Author,466072
1,Letters and Journals of Lord Byron,Author,260671
2,"Memoirs of the Life of Sir Walter Scott, Bart.",Subject,533282
3,Letters and Journals of Lord Byron,Subject,368442
4,"Memoirs of the Life of Sir Walter Scott, Bart.",Other,0
5,Letters and Journals of Lord Byron,Other,7496
